In [1]:
import findspark
findspark.init("/opt/spark")
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
.appName("market_retail") \
.config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
.config("spark.executer.memory","2g") \
.config("spark.driver.memory","1g") \
.master("local[2]") \
.enableHiveSupport() \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6e1644ae-0042-415c-830c-5f73255a8b3f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 558ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [3]:
#direkt olarak github'tan veri seti çekmek
from pyspark import SparkFiles
sc = spark.sparkContext
github_url = "https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/order_items.csv"
sc.addFile(github_url)
order_items = spark.read.csv(SparkFiles.get("order_items.csv"),header= True, inferSchema=True)
order_items.show(3)

+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|            1|               1|               957|                1|           299.98|               299.98|
|            2|               2|              1073|                1|           199.99|               199.99|
|            3|               2|               502|                5|            250.0|                 50.0|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
only showing top 3 rows



In [4]:
sc = spark.sparkContext
github_url = "https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/orders.csv"
sc.addFile(github_url)
orders = spark.read.csv(SparkFiles.get("orders.csv"),header= True, inferSchema=True)
orders.show(3)

+-------+-------------------+---------------+---------------+
|orderId|          orderDate|orderCustomerId|    orderStatus|
+-------+-------------------+---------------+---------------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|
|      3|2013-07-25 00:00:00|          12111|       COMPLETE|
+-------+-------------------+---------------+---------------+
only showing top 3 rows



In [5]:
sc = spark.sparkContext
github_url = "https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products.csv"
sc.addFile(github_url)
products = spark.read.csv(SparkFiles.get("products.csv"),header= True, inferSchema=True)
products.show(3)

+---------+-----------------+--------------------+------------------+------------+--------------------+
|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|
+---------+-----------------+--------------------+------------------+------------+--------------------+
|        1|                2|Quest Q64 10 FT. ...|              null|       59.98|http://images.acm...|
|        2|                2|Under Armour Men'...|              null|      129.99|http://images.acm...|
|        3|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
+---------+-----------------+--------------------+------------------+------------+--------------------+
only showing top 3 rows



In [6]:
sc = spark.sparkContext
github_url = "https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/categories.csv"
sc.addFile(github_url)
categories = spark.read.csv(SparkFiles.get("categories.csv"),header= True, inferSchema=True)
categories.show(3)

+----------+--------------------+-------------------+
|categoryId|categoryDepartmentId|       categoryName|
+----------+--------------------+-------------------+
|         1|                   2|           Football|
|         2|                   2|             Soccer|
|         3|                   2|Baseball & Softball|
+----------+--------------------+-------------------+
only showing top 3 rows



In [7]:
# Products ve Categories birleştirme.
cat_products = products.join(categories, products['productCategoryId'] == categories['categoryId']) \
.select("productId","productName", "categoryName")
cat_products.limit(5).toPandas().head()

,productId,productName,categoryName
0,1,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,Soccer
1,2,Under Armour Men's Highlight MC Football Clea,Soccer
2,3,Under Armour Men's Renegade D Mid Football Cl,Soccer
3,4,Under Armour Men's Renegade D Mid Football Cl,Soccer
4,5,Riddell Youth Revolution Speed Custom Footbal,Soccer


In [8]:
# orders ve order_items birleştirme
orders_and_items = order_items.join(orders, order_items['orderItemOrderId'] == orders['orderId']) \
.select("orderId","orderItemProductId","orderItemSubTotal","orderStatus")
orders_and_items.limit(5).toPandas().head()

,orderId,orderItemProductId,orderItemSubTotal,orderStatus
0,1,957,299.98,CLOSED
1,2,1073,199.99,PENDING_PAYMENT
2,2,502,250.00,PENDING_PAYMENT
3,2,403,129.99,PENDING_PAYMENT
4,4,897,49.98,CLOSED


In [9]:
# Ceategory-Products birleşimini iptal edilmiş siparişlerle ürün id üzerinden birleştirme
final_table = orders_and_items.join(cat_products, orders_and_items['orderItemProductId'] == cat_products['productId'])
final_table.limit(5).toPandas().head(5)

,orderId,orderItemProductId,orderItemSubTotal,orderStatus,productId,productName,categoryName
0,4,897,49.98,CLOSED,897,Team Golf New England Patriots Putter Grip,Accessories
1,21,897,74.97,PENDING,897,Team Golf New England Patriots Putter Grip,Accessories
2,312,897,74.97,COMPLETE,897,Team Golf New England Patriots Putter Grip,Accessories
3,697,897,49.98,COMPLETE,897,Team Golf New England Patriots Putter Grip,Accessories
4,1631,897,49.98,CLOSED,897,Team Golf New England Patriots Putter Grip,Accessories


In [12]:
#Toplam satış tutarı bakımından en çok iptal edilen ürünler
most_cancelled_products = final_table.filter("orderStatus == 'CANCELED'") \
.groupBy("productName").agg(F.sum("orderItemSubTotal").alias("Totalprice")) \
.orderBy(F.desc("TotalPrice"))
most_cancelled_products.limit(5).toPandas().head(5)

,productName,Totalprice
0,Field & Stream Sportsman 16 Gun Fire Safe,134393.28
1,Perfect Fitness Perfect Rip Deck,85785.70
2,Nike Men's Free 5.0+ Running Shoe,80691.93
3,Diamondback Women's Serene Classic Comfort Bi,80094.66
4,Pelican Sunstream 100 Kayak,66196.69


In [14]:
#parguet formatında kaydetmek
import time
start_time = time.time()
most_cancelled_products.write.parquet('/home/train/most_cancelled_products.parquet', mode='overwrite')
print("----------- %s secs ----------" % (time.time() - start_time))

----------- 7.207326173782349 secs ----------


In [17]:
#orderDate sütununu tarih formatına çevirmek
from pyspark.sql.functions import year, month, sum, date_format
orders = orders.withColumn("orderDate", orders["orderDate"].cast("timestamp"))

In [18]:
#dataframe Birleştirmek
join_df = orders.join(order_items, orders["orderId"] == order_items["orderItemOrderId"])
join_df.show(5)

+-------+-------------------+---------------+---------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|orderId|          orderDate|orderCustomerId|    orderStatus|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|
+-------+-------------------+---------------+---------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|            1|               1|               957|                1|           299.98|               299.98|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|            2|               2|              1073|                1|           199.99|               199.99|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|            3|               2|               502|                5|            250.0|   

In [19]:
#yıl ve ay bazında toplam satış miktarını hesaplamak ve gruplandırmak
ay_satis = join_df.withColumn("yil", year("orderDate")).withColumn("ay", month("orderDate")) \
.groupBy("yil", "ay").agg(sum("orderItemSubTotal").alias("toplamSatis"))
ay_satis.show()

+----+---+------------------+
| yil| ay|       toplamSatis|
+----+---+------------------+
|2014|  4| 2807789.799999905|
|2013|  9| 2934527.270000152|
|2013| 10| 2624600.610000026|
|2014|  5| 2753078.219999891|
|2013| 12| 2932964.270000029|
|2014|  1|  2924447.00999994|
|2013|  7|  764782.189999975|
|2014|  3| 2862492.209999953|
|2013| 11| 3168656.030000238|
|2014|  2| 2778663.659999922|
|2014|  6| 2703463.439999842|
|2013|  8|   2828658.7000001|
|2014|  7|2238496.5199998547|
+----+---+------------------+



In [20]:
max_satis = ay_satis.orderBy("toplamSatis", ascending=False).first()
en_yuksek_satis_yil = max_satis["yil"]
en_yuksek_satis_ay = max_satis["ay"]
print(f"En yüksek toplam satış {en_yuksek_satis_yil} yılının {en_yuksek_satis_ay}. ayında gerçekleşmiştir.")

En yüksek toplam satış 2013 yılının 11. ayında gerçekleşmiştir.


In [21]:
#orderDate sütununu gün formatına dönüştürmek
join_df2 = join_df.withColumn("orderDate", date_format("orderDate", "yyyy-MM-dd"))
join_df2 = join_df2.withColumn("gun", date_format("orderDate", "E"))

In [22]:
#haftanın günlerinde toplam ne kadar satış olduğunu hesaplamak ve gruplandırmak
haftalik_satis = join_df2.groupBy("gun").agg(sum("orderItemSubTotal").alias("toplamSatis"))
haftalik_satis.show()

+---+-----------------+
|gun|      toplamSatis|
+---+-----------------+
|Sun|4846988.500000717|
|Mon|4558745.380000613|
|Thu| 4977875.18000086|
|Sat|4946465.980000845|
|Wed|4904572.310000682|
|Fri|5172864.230001052|
|Tue|4915108.350000845|
+---+-----------------+



In [24]:
from pyspark.sql.functions import col
en_yuksek_satis = haftalik_satis.orderBy(col("toplamSatis").desc()).first()
gun_adi = {
    "Mon": "Pazartesi",
    "Tue": "Salı",
    "Wed": "Çarşamba",
    "Thu": "Perşembe",
    "Fri": "Cuma",
    "Sat": "Cumartesi",
    "Sun": "Pazar"
}
en_yuksek_satis_gun = gun_adi.get(en_yuksek_satis["gun"])
print(f"En yüksek toplam satış {en_yuksek_satis_gun} gününde gerçekleşmiştir.")

En yüksek toplam satış Cuma gününde gerçekleşmiştir.
